### Imports

In [1]:
import pandas as pd

### Input, Manipulation, and Output

In [2]:
folder_name = "BULK-AllRestaurantCat3StrictCalEquality/" # This must end in a slash
output_name = "All Restaurant Food Count by Age & Restaurant"
outname = "./ModelOutput/" + folder_name + output_name + ".xlsx"
multi_obj = False

print("Reading results...")
ages = [2, 4, 9, 14, 19, 31, 51]
genders = ["Male", "Female"]
objectives = []
foods = []
for age in ages:
    for gender in genders:
        in_name = f"{gender}-{age}"
        inname = "./ModelOutput/" + folder_name + in_name + ".xlsx" 
        ob = pd.read_excel(inname, sheet_name="Rankings")
        ob.loc[:, "Age"] = age
        ob.loc[:, "Gender"] = gender
        objectives.append(ob)
        
        fd = pd.read_excel(inname, sheet_name="Foods")
        fd.loc[:, "Age"] = age
        fd.loc[:, "Gender"] = gender
        foods.append(fd)
ob_bunch = pd.concat(objectives).rename({"Unnamed: 0": "Restaurant"}, axis=1)
fd_bunch = pd.concat(foods).rename({"Unnamed: 0": "Restaurant"}, axis=1)

# curr_row = 1
all_results = []
for res in pd.unique(fd_bunch["Restaurant"]):  # ["Blimpie"]
        print(res, end=", ")

        filtered_foods = fd_bunch[fd_bunch["Restaurant"] == res].loc[:, ["Food", "Objective", "Amount", "Gender", "Age"]]
        ptable = pd.pivot_table(filtered_foods, index=["Food", "Objective"], columns=["Gender", "Age"], observed=True).round(2)

        escape_amp = lambda s: s.replace("&", r"\&").replace("%", "\%")  # LaTeX does not like the raw '&'. This and the following line escape ('\&') them.
        ptable.index = pd.MultiIndex.from_tuples([tuple([escape_amp(l) for l in row]) for row in ptable.index])

        shorted_cals = lambda s: s.replace("Calories From Fat", "Fat")  # LaTeX does not like the raw '&'. This and the following line escape ('\&') them.
        ptable.index = pd.MultiIndex.from_tuples([tuple([shorted_cals(l) for l in row]) for row in ptable.index])

        ptable = ptable.T
        ptable.insert(0, "Restaurant", [res] + [""] * (len(ptable.index) - 1))
        ptable = ptable.T
        
        all_results.append(ptable)

full_table = pd.concat(all_results)
with pd.ExcelWriter(outname) as writer:
    full_table.to_excel(writer, merge_cells=False)
        
print("Finished!\nResults written to: ", outname)

Reading results...


ValueError: Worksheet named 'Rankings' not found

In [16]:
escape_percent = lambda s: str(s).replace("%", r"\%")  # LaTeX does not like the raw '&'. This and the following line escape ('\&') them.
tmp = pd.read_excel(outname, index_col=[0, 1])
tmp.index = pd.MultiIndex.from_tuples([tuple([escape_percent(l) for l in row]) for row in tmp.index])
tmp.to_excel(outname[:-5] + "-modified.xlsx", merge_cells=False)